<a href="https://colab.research.google.com/github/supportchelsea/Weather-Competition/blob/master/crawling_selenium_items_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# colab에서 실행시 반드시 선행할 것


In [0]:
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
Suggested packages:
  webaccounts-chromium-extension unity-chromium-extension adobe-flashplugin
The following NEW packages will be installed:
  chromium-browser chromium-browser-l10n chromium-chromedriver
  chromium-codecs-ffmpeg-extra
0 upgraded, 4 newly installed, 0 to remove and 4 not upgraded.
Need to get 70.1 MB of archives.
After this operation, 257 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-codecs-ffmpeg-extra amd64 75.0.3770.90-0ubuntu0.18.04.1 [1,112 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser amd64 75.0.3770

In [0]:
# !apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin
# !pip install selenium

In [0]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import datetime as dt
import urllib.parse
import re
from tqdm import tqdm_notebook
import pandas as pd
import sys
import numpy as np
import pickle

In [0]:
# for문 내 동적 변수 설정을 위한 코드
mod = sys.modules[__name__]

# 셀레니움을 위한 크롬 옵션 (max에러 발생시 여기서 부터 재실행)

In [0]:

options = webdriver.ChromeOptions()
options.add_argument ('--headless')
options.add_argument ('--no-sandbox')
options.add_argument ('--disable-dev-shm-usage')

browser = webdriver.Chrome ( 'chromedriver' , options = options)

In [0]:
# 트윗 클렌징을 위한 함수
def cleantweet(total_tweets_dict):
    keys = list(total_tweets_dict.keys())
    cleaned_tweet_dict = {}
  
    for key in keys:
        cleaned_tweet = []
        cleaned_tweet_dict[key] = []
        tweets_value = total_tweets_dict[key]
    
        for tweet in tweets_value:
            tmp = re.sub('<.+?>', '', str(tweet), 0).strip()
            tmp = re.sub('pic.+','',str(tmp),0).strip()
            tmp = re.sub('http?://\S+','',str(tmp),0).strip()
            tmp = re.sub(' https?://\S+','',str(tmp),0).strip()
            tmp = re.sub('\n','',str(tmp),0).strip()
            cleaned_tweet.append(tmp)

        cleaned_tweet_dict[key] += cleaned_tweet
    
    return cleaned_tweet_dict

In [0]:
# categories = [{'네일' : ['네일']}, {'립컬러' : ['틴트', '립스틱','립무스', '립펜슬' ]},
#               {'립케어' : ['립오일','립밤','립글로즈']}, 
#               {'마스크팩':['마스크팩']}, {'바디로션':['바디로션']},
#               {'선케어':['선크림','선블록','선밤','선파우더','선스프레이','선쿠션','썬크림','썬블록','썬밤','썬파우더','썬스프레이','썬쿠션']},
#               {'제모제':['제모제', '왁싱패치', '제모크림', '왁스스트립']},
#               {'체중조절':['체중조절', '다이어트간식' , '다이어트바']}, 
#               {'크림로션':['스킨', '토너', '로션', '에멀젼', '에센스', '세럼', '수분크림', '탄력크림', '아이크림']},
#               {'훼이셜클렌저':['클렌징', '아이리무버', '클렌저']}]

In [0]:
categories = {'네일' : ['네일'],
              '립컬러' : ['틴트', '립스틱','립무스', '립펜슬' ],
              '립케어' : ['립오일','립밤','립글로즈'], 
              '마스크팩':['마스크팩'], '바디로션':['바디로션'],
              '선케어':['선크림','선스틱','선쿠션','썬크림','썬스틱','썬쿠션'],
              '제모제':['제모제', '왁싱패치', '제모크림', '왁스스트립'],
              '체중조절':['체중조절', '다이어트간식' , '다이어트바'], 
              '크림로션':['스킨', '토너', '로션', '에센스','수분크림'],
              '훼이셜클렌저':['클렌징', '아이리무버', '클렌저']}

In [0]:
for category,kwds in tqdm_notebook(categories.items(),desc = '제품군 별'):
    daily_tweet = {}

    startdate=dt.date(year=2018,month=1,day=1)
    enddate = dt.date(year=2018,month=12,day=31)
        
    for i in np.arange(startdate,enddate):
        daily_tweet[str(i)] = []
    
    
    for kwd in tqdm_notebook(kwds,desc = '키워드 별'):
      
        keyword = urllib.parse.quote_plus(str(kwd))

      # 검색을 위한 변수 설정 
        startdate=dt.date(year=2018,month=1,day=1)
        untildate=dt.date(year=2018,month=1,day=2)
            
        while not enddate==startdate:            

            url='https://twitter.com/search?l=ko&q='+keyword+'%20since%3A'+str(startdate)+'%20until%3A'+str(untildate)+'&src=typd'

            browser.get (url)

            html = browser.page_source
            soup=BeautifulSoup(html,'html.parser')

            lastHeight = browser.execute_script("return document.body.scrollHeight")
 
            while True:
                tweets=soup.find_all("p", {"class": "TweetTextSize"})

                browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
 
                newHeight = browser.execute_script("return document.body.scrollHeight")
                
                if newHeight != lastHeight:
                    daily_tweet[str(startdate)] += tweets
                else:
                    daily_tweet[str(startdate)] += tweets
                    startdate=untildate
                    untildate+=dt.timedelta(days=1)
                    break

                lastHeight = newHeight
                       
            if startdate.month != untildate.month:
                print(startdate.month)

        setattr(mod, 'var_{}'.format(category), cleantweet(daily_tweet))

1
2
3
4
5
6
7
8
9
10
11
12


1
2
3
4
5
6
7
8
9
10
11
12
1
2
3
4
5
6
7
8
9
10
11
12
1
2
3
4
5
6
7
8
9
10
11
12
1
2
3
4
5
6
7
8
9
10
11
12


1
2
3
4
5
6
7
8
9
10
11
12
1
2
3
4
5
6
7
8
9
10
11
12
1
2
3
4
5
6
7
8
9
10
11
12


1
2
3
4
5
6
7
8
9
10
11
12


1
2
3
4
5
6
7
8
9
10
11
12


1
2
3
4
5
6
7
8
9
10
11
12
1
2
3
4
5
6
7
8
9
10
11
12
1
2
3
4
5
6
7
8
9
10
11
12
1
2
3
4
5
6
7
8
9
10
11
12
1
2
3
4
5
6
7
8
9
10
11
12
1
2
3
4
5
6
7
8
9
10
11
12


1
2
3
4
5
6
7
8
9
10
11
12
1
2
3
4
5
6
7
8
9
10
11
12
1
2
3
4
5
6
7
8
9
10
11
12
1
2
3
4
5
6
7
8
9
10
11
12


1
2
3
4
5
6
7
8
9
10
11
12
1
2
3
4
5
6
7
8
9
10
11
12
1
2
3
4
5
6
7
8
9
10
11
12


1
2
3
4
5
6
7
8
9
10
11
12
1
2
3
4
5
6
7
8
9
10
11
12
1
2
3
4
5
6
7
8
9
10
11
12
1
2
3
4
5
6
7
8
9
10
11
12
1
2
3
4
5
6
7
8
9
10
11
12


1
2
3
4
5
6
7
8
9
10
11
12
1
2
3
4
5
6
7
8
9
10
11
12
1
2
3
4
5
6
7
8
9
10
11
12



In [0]:
len(var_네일)

364

In [0]:
nail_2018 = pd.DataFrame.transpose(pd.DataFrame.from_dict(var_네일,orient='index'))
lipcolor_2018 = pd.DataFrame.transpose(pd.DataFrame.from_dict(var_립컬러,orient='index'))
lipcare_2018 = pd.DataFrame.transpose(pd.DataFrame.from_dict(var_립케어,orient='index'))
mask_2018 =  pd.DataFrame.transpose(pd.DataFrame.from_dict(var_마스크팩,orient='index'))
bodylotion_2018 =  pd.DataFrame.transpose(pd.DataFrame.from_dict(var_바디로션,orient='index'))
suncare_2018 =  pd.DataFrame.transpose(pd.DataFrame.from_dict(var_선케어,orient='index'))
hairremoval_2018 = pd.DataFrame.transpose(pd.DataFrame.from_dict(var_제모제,orient='index'))
diet_2018 = pd.DataFrame.transpose(pd.DataFrame.from_dict(var_체중조절,orient='index'))
creamlotion_2018 = pd.DataFrame.transpose(pd.DataFrame.from_dict(var_크림로션,orient='index'))
cleanser_2018 = pd.DataFrame.transpose(pd.DataFrame.from_dict(var_훼이셜클렌저,orient='index'))


In [0]:
nail_2018.head()

,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08,2018-01-09,2018-01-10,2018-01-11,2018-01-12,2018-01-13,2018-01-14,2018-01-15,2018-01-16,2018-01-17,2018-01-18,2018-01-19,2018-01-20,2018-01-21,2018-01-22,2018-01-23,2018-01-24,2018-01-25,2018-01-26,2018-01-27,2018-01-28,2018-01-29,2018-01-30,2018-01-31,2018-02-01,2018-02-02,2018-02-03,2018-02-04,2018-02-05,2018-02-06,2018-02-07,2018-02-08,2018-02-09,...,2018-11-21,2018-11-22,2018-11-23,2018-11-24,2018-11-25,2018-11-26,2018-11-27,2018-11-28,2018-11-29,2018-11-30,2018-12-01,2018-12-02,2018-12-03,2018-12-04,2018-12-05,2018-12-06,2018-12-07,2018-12-08,2018-12-09,2018-12-10,2018-12-11,2018-12-12,2018-12-13,2018-12-14,2018-12-15,2018-12-16,2018-12-17,2018-12-18,2018-12-19,2018-12-20,2018-12-21,2018-12-22,2018-12-23,2018-12-24,2018-12-25,2018-12-26,2018-12-27,2018-12-28,2018-12-29,2018-12-30
0,아무튼 시스루 셔츠에 블링블링 목걸이 여러개 레이어드하고 반지 잔뜩 낀 손에 까만네...,헉 쑤님 네일 먼가 저랑 정반대같아요 레드골드 그린실버,네일 붙였다 만족 신난다 울루,네일을 풀로 그리고 진한색으로 함이유: 사진이 말해줌ㅎ머리감고나면 손이 파란색으로 ...,아 호넷 서치하다 호넷 방 오브젝트 봤는데 어흐흑 너무 조와.......바늘꽂이에 ...,이타치 현대 AU에서도 매니큐어 발라주면 제가 좋아 죽거든요. 검은 터틀넥에 간 ...,어제밤에 셀프네일하고 잤다.,연탄이랑 타타 네일 샘플 했어여~ 2018년 황금개띠 해 연탄 네일 해야지 Did...,6콘보는데 아이 손에 깃털 네일 볼때마다 렌 간지럽히고싶었어서 못참고 아이렌으로 ....,아! 저 원래 손톱 물어뜯는 습관이 엄청 심해서 매번 피나고 아파도 계속 물어뜯었는...,훙냥냥 오랜만애 네일햇댱 잠잘고임 ㅠ (젤 안흔들린 사진 올린거 맞음,네일깅ㅋㅋㅋㅋㅋㅋ않이 근데 옹깅이 속눈썹 넘무 부러옹 ༼;´༎ຶ ۝ ༎ຶ༽ ༼;´༎...,잠은 안오고 요상한 네일이나 해버렸곸ㅋㅋㅋㅋ나능 손톱에 뭘 하고 싶었던걸까?,내가 가는 네일샵 사장님이 가상화폐 8000만원 넣었다 했을때... 뭐지? 했다. ...,우리 쟈근기사님이 감염된 선대기사님을 드림네일 써서 통로로 활용해 광휘를 만나러 가...,처음으로 젤네일을 해서 꽤 오래 손톱을 기르고 있는데 느끼는 건 나는 취미 생활 접...,않이 거지같은 젤네일 놈들 쌍으로 탈주하더니 이 새낀 또 어디로 빤스런 한 거임 ㅡㅡ,완 네일 넘예다,그거 생각난다 작년에수련회때 네일리무버냄새라고 속이고술마셨던겈ㅋㅋㅋㅋㅋ,"조금 번졌지만 ,,,, 네일 하다가 시간 다 지났다 ㅋㅋㅋㅋㅋ 내 폰 카메라가 안 ...",전희진 손목 손가락 피어싱 주렁주렁 달고다니는데ㅜ네일 안하고 셀메도못하는거귀엽다,흡혈귀 시착햇다 히히S사이즈인데 나름 프리한듯? 속상의는 아슬아슬하당오른손 네일만 ...,"네일이름은 하트보내기 잊지않기,,,☆",제가정치하시는분들께부탁하고십던그런일을안대표님께서홀현히들고나왔습니다.이번일이얼마나힘들...,이 재료는 막 얹어도 참으로 예쁨 백매같기도 다이아같기도이건 딸기색 투명네일 위에 ...,복숭쓰 콜라 별루임 (속마음: 저의 새로운 네일 귀엽죠 귀엽다고 말해 빨리)#iHe...,이쁘죠 네일 바꿨어요,제가 드디어 네일을 바꿧습니다네일로 까지 덕질을 하는 접니다....,뉴 네일 해따,일베충이 만들었다는 평창유감 기사 검색 중 발견한 사실.기사 제목이 제일 더러운 기...,언다인(Undyne). 노멀룩. 생선뼈 모양이 그려진 스티커는 네일건에도 붙어있고 ...,네일함,셀프 젤네일! 저번 네일 엉망이 되어서 새로했다!,히데오 성우분 프레임 네일 대박 오졋고 너무 좋앗음... 그리고 한쪽팔은 내의가 길...,계속 지켜보는데순간적으로 12위로 갔다가 다시 2위로 돌아오는신적인 기술을 네일베가...,세상에 네일했는데 아이스크림 먹으니까 이렇거 됐어 아 #iHeartAwards #B...,"기타 쳐서 왼손은 짧게, 오른손은 길게... 그래서 네일도 못하는..슬픈..쥰이.....",#보검투어_뉴질랜드 준비편어제 팬미팅 사전신청 멘붕으로 오늘 아침에야 올리는 여행 ...,잠 안자려고 앨범 뒤지다가 .. 내가 햇던 네일중에 재일 오졋던거 세개 ㅋㅋㅋ ㅋㅌ...,3.티르코네일 던컨댁에는 이상한 새가 살고있다. 이 파란새의 털을뽑아 밀레시안의 부...,...,동재가 시목이 손톱 깎아주는거 ㅂㄱㅅㄷ 심옥.. 왠지 피볼거 같아 싫다고 하다가 동...,어데 받은 세상 화려 네일,여러분 제 네일을 봐죠,금학현수 덕톡회 갔다왔서요. 샘들이 예쁜 굿즈 왕 마니 주셧다 따콩샘이 네일도 해주...,작중에 등장할 칵테일 러스티 네일입니다,와나...졸업시험이 합격을 하던 해야지 네일 커미션을 위해 디자인 공부하던 할텐데 ...,네일 받고 와쏘-!-!/! 2만원 바께 안해ㅠㅠ 너무 싼데 너무 이뻐ㅠ,1.(아마도)반짝이 뱀파이어 메이크업피부_루나 롱래스팅 파데렌즈_오렌즈 뱀파이어렌즈...,네일샷 같은 마카롱선물인증샷마딛다 마딛어 @_memeda_,새벽의 마감쓰와 썸ㅁ네일,오늘의 네일,크리스마스 네일 진쨔 너모 예쁜 거 아냐 ㅠㅁ ㅠ~~~ 머할찌 고민한지 300년이 지낫어,이그형 네일 검은색 검은색검은색검은색,그리고 콤마랑 세평이 마주친거보면 얘네 아파트 앞에 편의점이 문세평 근무지고 동시에...,지금 알았는데하스영 요 셀카에서 네일컬러 연보라색이야 . . . .연보라색 발랐어 ...,여행용 (임시)네일,첨으루 붓 들고 셀프네일 했는데 어때욥,일주일만에 바꾼 뉴 네일...,전트로이검은네일악개입니다,181210 미나 네일#MINA #미나 #名井南 #ミナ#超絶すぎる名井様,소문이 나겠지. 밀레시안 중 밀레시안이라는 그 분께 연인이 생겼대요! 타라에서 이멘...,181207 인천공항 출국 예뻐라네일~~~ #정화 #박정화 #쫑쫑이#jeonghw...,아 딤사님짤ㅋㅋㅋㅋㅋ넷플 파티셰를 잡아라 출연하는 박짐 보고싶어졌어...케익의 흔적...,네일 했다!!! 드디어 끝났다!!!!,G23 초반 티르코네일 연출 너무너무 맘에 들었다 ○○○○○○○○○○○○○○○○○○...,♡ 181212 이창섭 고민은 네일해 옷 정보 ♡제품명 : 로고 다이아 브이넥 니트...,발리는포인트1.키차이 2.토도리야 빌뎈캐입 완전하신 표정3.조반아 네일 블랙임 ㄹㅇ...,181215 강남 영풍 팬사인회 이창섭 후기( @LeeCS_BTOB ) 안녕 사랑아...,카카시 시한부인생 각재면 덤덤히 받아들이고 주변사람들한테는 죽어도 안알릴것같음 오히...,아니알티너무웃기고공감내호모 온리전 갈 때 : 꺄 드레스코드 레트로래여 님 머 입을거...,네일데칼 직접뽑아서 햇다..... ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㄱㅋ 신기해,"크리스마스 특집 네일. 손톱끝에 뭔가 발라 놓으니 배고플 때, 스트레스 받았을 때 ...",아 쉬바 네일 자랑할까? 하면서 벽에 손 대고 플레쉬 터트리고 있다가 거미 잡음,버티다 버티다 젤 네일 새로 받음 ㅎㅎ검지 포인트 펄은 잘 안보이네 .. 크리스마스...,완이 네일 새로 했니https://www.instagram.com/p/Br1Eprx...,꼬까네일인데 어떻게 찍어도 원래 색이 안나오는 신비... 자주랑 청록인데 카메라 왜...,양손잡이었으면 좋겠다 오른손 네일두 잘하고 싶어..,전 이 네일 두가지를 어래인지 할것...! :)호피무늬를 입지는 못하니.... 손톱...,네일샵가서

In [0]:
# nail_2018 = pd.DataFrame.from_dict(var_네일,orient='index').transpose()
# lipcolor_2018 = pd.DataFrame.from_dict(var_립컬러,orient='index').transpose()
# lipcare_2018 = pd.DataFrame.from_dict(var_립케어,orient='index').transpose()
# mask_2018 =  pd.DataFrame.from_dict(var_마스크팩,orient='index').transpose()
# bodylotion_2018 =  pd.DataFrame.from_dict(var_바디로션,orient='index').transpose()
# suncare_2018 =  pd.DataFrame.from_dict(var_선케어,orient='index').transpose()
# hairremoval_2018 = pd.DataFrame.from_dict(var_제모제,orient='index').transpose()
# diet_2018 = pd.DataFrame.from_dict(var_체중조절,orient='index').transpose()
# creamlotion_2018 = pd.DataFrame.from_dict(var_크림로션,orient='index').transpose()
# cleanser_2018 = pd.DataFrame.from_dict(var_훼이셜클렌저,orient='index').transpose()

In [0]:
nail_2018.to_csv("nail_2018.csv")
lipcolor_2018.to_csv("lipcolor_2018.csv")
lipcare_2018.to_csv("lipcare_2018.csv")
mask_2018.to_csv("mask_2018.csv")
bodylotion_2018.to_csv("bodylotion_2018.csv")
suncare_2018.to_csv("suncare_2018.csv")
hairremoval_2018.to_csv("hairremoval_2018.csv")
diet_2018.to_csv("diet_2018.csv")
creamlotion_2018.to_csv("creamlotion_2018.csv")
cleanser_2018.to_csv("cleanser_2018.csv")


In [0]:
from google.colab import files

In [0]:
files.download('nail_2018.csv')
files.download('lipcolor_2018.csv')
files.download('lipcare_2018.csv')
files.download('mask_2018.csv')
files.download('bodylotion_2018.csv')
files.download('suncare_2018.csv')
files.download('hairremoval_2018.csv')
files.download('diet_2018.csv')
files.download('creamlotion_2018.csv')
files.download('cleanser_2018.csv')

In [0]:

# 립컬러
# 립케어
# 마스크팩
# 바디로션
# 선케어
# 제모제
# 체중조절
# 크림로션
# 훼이셜크림

In [0]:
# !cp nail_2018.csv ./gdrive/My\ Drive/GitHub/Weather\ Competition/
# !cp lipcolor_2018.csv ./gdrive/My\ Drive/GitHub/Weather\ Competition/
# !cp lipcare_2018.csv ./gdrive/My\ Drive/GitHub/Weather\ Competition/
# !cp mask_2018.csv ./gdrive/My\ Drive/GitHub/Weather\ Competition/
# !cp bodylotion_2018.csv ./gdrive/My\ Drive/GitHub/Weather\ Competition/
# !cp suncare_2018.csv ./gdrive/My\ Drive/GitHub/Weather\ Competition/
# !cp hairremoval_2018.csv ./gdrive/My\ Drive/GitHub/Weather\ Competition/
# !cp diet_2018.csv ./gdrive/My\ Drive/GitHub/Weather\ Competition/
# !cp creamlotion_2018.csv ./gdrive/My\ Drive/GitHub/Weather\ Competition/
# !cp cleanser_2018.csv ./gdrive/My\ Drive/GitHub/Weather\ Competition/

In [0]:
# !cp lalavla_2018.csv ./gdrive/My\ Drive/GitHub/Weather\ Competition/

In [0]:
# import pickle

# with open('2018_lalavla.txt', 'wb') as f:
#      pickle.dump(cleaned_tweet_bag, f)

In [0]:
# with open('2018_lalavla.txt', 'rb') as f:
# ...     data = pickle.load(f) # 단 한줄씩 읽어옴